<a href="https://colab.research.google.com/github/ti-molodykh/colab/blob/main/%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 3

## Оконные функции

In [ ]:
!pip install psycopg2

import psycopg2
import pandas as pd
# доступны student-ы от 0 до 9

DB_HOST = '89.223.95.138'
DB_USER = 'student_0'
DB_USER_PASSWORD = 'student_0'
DB_NAME = 'sample_db'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Материал из документации доступен [по ссылке](https://postgrespro.ru/docs/postgrespro/9.5/tutorial-window#:~:text=%D0%9E%D0%BA%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F%20%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F%20%D0%B2%D1%8B%D0%BF%D0%BE%D0%BB%D0%BD%D1%8F%D0%B5%D1%82%20%D0%B2%D1%8B%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%20%D0%B4%D0%BB%D1%8F,%D0%BE%D0%B4%D0%BD%D1%83%2C%20%D0%B0%20%D0%BF%D1%80%D0%BE%D0%B4%D0%BE%D0%BB%D0%B6%D0%B0%D1%8E%D1%82%20%D1%81%D1%83%D1%89%D0%B5%D1%81%D1%82%D0%B2%D0%BE%D0%B2%D0%B0%D1%82%D1%8C%20%D0%BE%D1%82%D0%B4%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE.).

Оконная функция выполняет вычисления для набора строк, некоторым образом связанных с текущей строкой. Можно сравнить её с агрегатной функцией, но, в отличие от обычной агрегатной функции, при использовании оконной функции несколько строк не группируются в одну, а продолжают существовать отдельно. Внутри же, оконная функция, как и агрегатная, может обращаться не только к текущей строке результата запроса.

Рассмотрим основные оконные функции, представленные в postgresql.

Оконные функции могут быть применены в проекции (в части запроса SELECT) либо в части ORDER BY, но не в GROUP BY, HAVING и WHERE. Агрегатные функции выполняются раньше, чем оконные, и поэтому можно вызвать агрегатную функцию внутри оконной, но не наоборот.

Общий синтаксис:

```
SELECT
  col1,
  col2,
  window_function(arg1, arg2,..) OVER (
    [PARTITION BY partition_expression]
    [ORDER BY sort_expression [ASC | DESC] [NULLS {FIRST | LAST }])
FROM
  ...
```




### row_number()
Порядковый номер строки. Начнем с простых примеров. Для начала в выражении OVER оставим пустое окно. Тогда оконная функция будет применена ко всей таблице (грубо говоря мы имеем одно окно, состоящее из все таблицы). В данном случае мы просто добавили ешё один столбик, который содержит в себе порядковый номер каждой строки в ответе.

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       row_number() OVER ()
FROM employee
""", conn)

,employee_id,salary,row_number
0,7369,800.0,1
1,7499,1600.0,2
2,7505,2850.0,3
3,7506,2750.0,4
4,7507,2200.0,5
5,7521,1250.0,6
6,7555,1250.0,7
7,7557,1250.0,8
8,7560,1250.0,9
9,7564,1250.0,10


Теперь, если мы добавим сортировку в выражения для определения окна, то порядковые номера будут распределены в порядке зарплаты (salary).

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       row_number() OVER (ORDER BY salary ASC)
FROM employee
""", conn)

,employee_id,salary,row_number
0,7950,750.0,1
1,7954,750.0,2
2,7369,800.0,3
3,7919,800.0,4
4,7900,950.0,5
5,7876,1100.0,6
6,7555,1250.0,7
7,7557,1250.0,8
8,7560,1250.0,9
9,7564,1250.0,10


Теперь, наконец, перейдем к более содержательным окнам. В этот раз оконная функция будет применена по группам с одинаковой зарплатой. Отметим также, что внутри этих групп номера расставляются в порядке employee_id (по возрастанию).

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       row_number() OVER (PARTITION BY salary ORDER BY employee_id ASC)
FROM employee
""", conn)

,employee_id,salary,row_number
0,7950,750.0,1
1,7954,750.0,2
2,7369,800.0,1
3,7919,800.0,2
4,7900,950.0,1
5,7876,1100.0,1
6,7521,1250.0,1
7,7555,1250.0,2
8,7557,1250.0,3
9,7560,1250.0,4


Вот так мы можем взять, например, каждую первую запись в группе.

In [ ]:
pd.read_sql_query("""
SELECT *
FROM
  (SELECT employee_id,
        salary,
        row_number() OVER (PARTITION BY salary ORDER BY employee_id ASC) AS rn
  FROM employee) AS t1
WHERE
  t1.rn = 1
""", conn)

,employee_id,salary,rn
0,7950,750.0,1
1,7369,800.0,1
2,7900,950.0,1
3,7876,1100.0,1
4,7521,1250.0,1
5,7820,1300.0,1
6,7789,1500.0,1
7,7499,1600.0,1
8,7609,1800.0,1
9,7676,1850.0,1


### Мы можем использовать и привычные агрегатные (агрегационные) функции
Например, для подсчета средней зарплаты в отделе мы могли бы использовать и оконные функции. Обратите внимание, что в сравнении с применением GROUP BY все строки остаются на месте. Т.е. оконные функции только дополняют информацию в таблице. В то время как при использовании GROUP BY, мы обычно "собираем" (аггрегируем) информацию. 

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       AVG(salary) OVER (PARTITION BY department_id) AS average
FROM employee
""", conn)

,employee_id,salary,department_id,average
0,7934,1300.0,10,2916.666667
1,7782,2450.0,10,2916.666667
2,7839,5000.0,10,2916.666667
3,7916,2875.0,12,2418.750000
4,7569,3000.0,12,2418.750000
5,7919,800.0,12,2418.750000
6,7799,3000.0,12,2418.750000
7,7600,1250.0,13,1470.000000
8,7950,750.0,13,1470.000000
9,7555,1250.0,13,1470.000000


Обратим внимание на одну особенность. Рассмотрите результат работы этого запроса. Мы инкрементально увеличиваем в группе значение суммы. НО! Посмотрите на группу 1250 по зарплате. Мы видим 750 -> 4500 -> 4500 -> 4500. Для всех повторяющихся значений сумма сразу посчиталась для всех строк с одним значением, а не постепенно увеличивалась.

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       SUM(salary) OVER (PARTITION BY department_id ORDER BY salary ASC) AS something
FROM employee
""", conn)

,employee_id,salary,department_id,something
0,7934,1300.0,10,1300.0
1,7782,2450.0,10,3750.0
2,7839,5000.0,10,8750.0
3,7919,800.0,12,800.0
4,7916,2875.0,12,3675.0
5,7799,3000.0,12,9675.0
6,7569,3000.0,12,9675.0
7,7950,750.0,13,750.0
8,7600,1250.0,13,4500.0
9,7557,1250.0,13,4500.0


Для решения этой проблемы просто добавим каждой строке уникальности, например, добавив столбец с номером строки row_number(). Теперь мы можем, например, посчитать кумулятивную сумму в каждой группе.

Внимательно сравните результаты этого и предыдущего запросов.

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       AVG(salary) OVER (PARTITION BY department_id ORDER BY salary, rn ASC) AS cumulative_avg,
       SUM(salary) OVER (PARTITION BY department_id ORDER BY salary, rn ASC) AS cumulative_sum
FROM 
  (SELECT employee_id,
        salary,
        department_id,
        row_number() OVER (PARTITION BY department_id ORDER BY salary ASC) AS rn
  FROM employee) AS rn_tbl
""", conn)

,employee_id,salary,department_id,cumulative_avg,cumulative_sum
0,7934,1300.0,10,1300.000000,1300.0
1,7782,2450.0,10,1875.000000,3750.0
2,7839,5000.0,10,2916.666667,8750.0
3,7919,800.0,12,800.000000,800.0
4,7916,2875.0,12,1837.500000,3675.0
5,7799,3000.0,12,2225.000000,6675.0
6,7569,3000.0,12,2418.750000,9675.0
7,7950,750.0,13,750.000000,750.0
8,7600,1250.0,13,1000.000000,2000.0
9,7557,1250.0,13,1083.333333,3250.0


Чтобы избежать дублирование кода, пользуемся специальным синтаксисом

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       AVG(salary) OVER w AS cumulative_avg,
       SUM(salary) OVER w AS cumulative_sum
FROM 
    (SELECT employee_id,
        salary,
        department_id,
        row_number() OVER (PARTITION BY department_id ORDER BY salary ASC) AS rn
  FROM employee) AS rn_tbl
WINDOW w AS (PARTITION BY department_id ORDER BY salary, rn ASC) 
""", conn)

,employee_id,salary,department_id,cumulative_avg,cumulative_sum
0,7934,1300.0,10,1300.000000,1300.0
1,7782,2450.0,10,1875.000000,3750.0
2,7839,5000.0,10,2916.666667,8750.0
3,7919,800.0,12,800.000000,800.0
4,7916,2875.0,12,1837.500000,3675.0
5,7799,3000.0,12,2225.000000,6675.0
6,7569,3000.0,12,2418.750000,9675.0
7,7950,750.0,13,750.000000,750.0
8,7600,1250.0,13,1000.000000,2000.0
9,7557,1250.0,13,1083.333333,3250.0


### rank()
В отличии от row_number(), rank() присвоит одинаковым значениям в группе одно и то же число. См. результаты запроса ниже:

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       rank() OVER w AS salary_ranking,
       row_number() OVER w AS salary_rn
FROM employee
WINDOW w AS (PARTITION BY department_id ORDER BY salary ASC)
""", conn)

,employee_id,salary,department_id,salary_ranking,salary_rn
0,7934,1300.0,10,1,1
1,7782,2450.0,10,2,2
2,7839,5000.0,10,3,3
3,7919,800.0,12,1,1
4,7916,2875.0,12,2,2
5,7799,3000.0,12,3,3
6,7569,3000.0,12,3,4
7,7950,750.0,13,1,1
8,7600,1250.0,13,2,2
9,7557,1250.0,13,2,3


### dense_rank()
dense_rank() сделает практически то же самое, что и rank(), но обратите внимание на то как будут обработаны строки с одинаковыми значениями.

rank() ранг строки в пределах партиции с пропусками индексов (номера идут не подряд 1 1 3 4 4 6 … ). Нет аргументов, так как поведение полностью регулируется определением окна.

dense_rank()	 ранг строки в пределах партиции без пропусков индексов (т.е номера идут подряд 1 1 2 3 3 4 … ). Нет аргументов, так как поведение полностью регулируется определением окна.

In [ ]:
pd.read_sql_query("""
SELECT employee_id,
       salary,
       department_id,
       rank() OVER w AS salary_ranking,
       dense_rank() OVER w AS salary_dense_ranking,
       row_number() OVER w AS salary_rn
FROM employee
WINDOW w AS (PARTITION BY department_id ORDER BY salary ASC)
""", conn)

,employee_id,salary,department_id,salary_ranking,salary_dense_ranking,salary_rn
0,7934,1300.0,10,1,1,1
1,7782,2450.0,10,2,2,2
2,7839,5000.0,10,3,3,3
3,7919,800.0,12,1,1,1
4,7916,2875.0,12,2,2,2
5,7799,3000.0,12,3,3,3
6,7569,3000.0,12,3,3,4
7,7950,750.0,13,1,1,1
8,7600,1250.0,13,2,2,2
9,7557,1250.0,13,2,2,3


### lag() / lead()
Методы lag() и lead() позволяют нам посмотреть в соседние строки в результирующей таблице, что дает нам некоторую функциональность, близкую к обычным языкам программирования общего назначения.


`lag(value anyelement [, offset integer [, default anyelement ]])` Вычисляет выражение value по строке предшествующей текущей рассматриваемой на расстоянии offset в рамках партиции. Если такой строки нет, то вычисляется выражение в default.

lead то же что lag, но в другую сторону


Посмотрим как изменялась объявленная стоимость на продукты

In [ ]:
pd.read_sql_query("""
SELECT product_id,
       start_date,
       end_date,
       list_price,
       list_price - lag(list_price, 1, 0.0) OVER w AS delta
FROM price
WINDOW w AS (PARTITION BY product_id ORDER BY start_date ASC)
""", conn)

,product_id,start_date,end_date,list_price,delta
0,100860,1989-01-01,1989-12-31,30.0,30.0
1,100860,1990-01-01,1990-05-31,32.0,2.0
2,100860,1990-06-01,NaT,35.0,3.0
3,100861,1989-01-01,1989-12-31,39.0,39.0
4,100861,1990-01-01,1990-05-31,42.0,3.0
5,100861,1990-06-01,NaT,45.0,3.0
6,100870,1989-01-01,1989-12-01,2.4,2.4
7,100870,1990-01-01,NaT,2.8,0.4
8,100871,1989-01-01,1989-12-01,4.8,4.8
9,100871,1990-01-01,NaT,5.6,0.8


### Задание
Вывести наименьший срок доставки товара для каждого customer_id.

* Дата заказа order_date
* Дата поставки ship_date

Понадобятся функции age, date_part (вывести разницу в днях), а также оконная функция row_number либо rank (dense_rank).

Можно ограничиться запросом только к sales_order, т.е не соединять с таблицей customer (выводим только customer_id).

In [ ]:
pd.read_sql_query("""
SELECT  customer_id,
        delta_days,
        rn
FROM
  (SELECT  customer_id,
          age(ship_date, order_date) AS delta_days,
          row_number() OVER (PARTITION BY customer_id ORDER BY age(ship_date, order_date) ASC) AS rn
  FROM sales_order) AS ranking
WHERE ranking.rn = 1
""", conn)

,customer_id,delta_days,rn
0,100,0 days,1
1,101,1 days,1
2,102,0 days,1
3,103,7 days,1
4,104,0 days,1
5,105,26 days,1
6,106,15 days,1
7,107,5 days,1
8,108,0 days,1
9,201,2 days,1


In [ ]:
pd.read_sql_query("""
SELECT  customer_id,
        delta_days
FROM
  (SELECT  customer_id,
          age(ship_date, order_date) AS delta_days,
          row_number() OVER (PARTITION BY customer_id ORDER BY age(ship_date, order_date) ASC) AS rn
  FROM sales_order) AS ranking
WHERE ranking.rn = 1
ORDER BY delta_days ASC
""", conn)

,customer_id,delta_days
0,223,0 days
1,108,0 days
2,102,0 days
3,100,0 days
4,104,0 days
5,226,0 days
6,224,0 days
7,228,1 days
8,101,1 days
9,204,1 days


Данная задача дана в виде примера, в принципе то же самое мы могли бы получить и без оконных функций:

In [ ]:
pd.read_sql_query("""
SELECT
  customer_id,
  MIN(age(ship_date, order_date)) AS min_delta_days
FROM sales_order
GROUP BY customer_id
ORDER BY MIN(age(ship_date, order_date))
""", conn)

,customer_id,min_delta_days
0,224,0 days
1,226,0 days
2,104,0 days
3,108,0 days
4,102,0 days
5,100,0 days
6,223,0 days
7,228,1 days
8,101,1 days
9,204,1 days


Для того, чтобы ближе познакомиться с этим функционалом, рекомендуется порешать задачки на эту тему.